<a href="https://colab.research.google.com/github/swarathesh/COVID_CSGAN_WEBAPP/blob/master/Deep_Convolutional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Generative Adversarial Network

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/dataset/

/content/drive/My Drive/dataset


##### Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from time import time
import os
import pandas as pd
# os.chdir('/content/drive/My Drive/dataset/')
import argparse
import math
import re
import itertools
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

##### Loading all file names

In [ ]:
files = glob('/content/drive/My Drive/dataset/co/chest_xray/test/COVID19 /*')

##### Loading the covid x-ray images


In [ ]:
def crop(img):
    if img.shape[0]<img.shape[1]:
        x = img.shape[0]
        y = img.shape[1]
        crop_img = img[: , int(y/2-x/2):int(y/2+x/2)]
    else:
        x = img.shape[1]
        y = img.shape[0]
        crop_img = img[int(y/2-x/2):int(y/2+x/2) , :]

    return crop_img

In [ ]:
import glob
filelist = glob.glob('/content/drive/My Drive/dataset/co/chest_xray/test/covid19/*')
dim = 60
x = []
for ix, file in enumerate(filelist): 
    image = plt.imread(file)
    image = Image.fromarray((image * 255).astype(np.uint8)).resize((dim, dim)).convert('L')
    image = crop(np.array(image))
    x.append(image)
y = np.repeat(1, len(x))

##### pictures into arrays

In [ ]:
x = np.divide(np.array(x, dtype=np.float32), 255).reshape(-1, 1, 60, 60)
y = np.array(y, dtype=np.float32)

In [ ]:
x.shape,y.shape

((234, 1, 60, 60), (234,))

##### targets into a 2D matrix

In [ ]:
print('Scaling...', end='')
image_size = x.shape[1] * x.shape[1]
print('\rDone.     ')

Done.     


#####  tensors to CUDA

In [ ]:
if torch.cuda.is_available():
    x = torch.from_numpy(x) 
    y = torch.from_numpy(y)
    print('Tensors successfully flushed to CUDA.')
else:
    print('CUDA not available!')

Tensors successfully flushed to CUDA.


##### Making a dataset class

In [ ]:
class covid():
    
    def __init__(self):
        self.len = x.shape[0]
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return x[index], y[index].unsqueeze(0) 
    
    def __len__(self):
        return self.len

##### Instantiating the class

In [ ]:
data = covid()

##### Parsing the args

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=5_000)
parser.add_argument("--batch_size", type=int, default=32)
parser.add_argument("--lr", type=float, default=0.001)
parser.add_argument("--b1", type=float, default=0.3)
parser.add_argument("--b2", type=float, default=0.999)
parser.add_argument("--n_cpu", type=int, default=8)
parser.add_argument("--latent_dim", type=int, default=128)
parser.add_argument("--img_size", type=int, default=60)
parser.add_argument("--channels", type=int, default=1)
parser.add_argument("--sample_interval", type=int, default=5)
opt, unknown = parser.parse_known_args()
print(opt)

Namespace(b1=0.3, b2=0.999, batch_size=32, channels=1, img_size=60, latent_dim=128, lr=0.001, n_cpu=8, n_epochs=5000, sample_interval=5)


In [ ]:
cuda = True if torch.cuda.is_available() else False

##### Initializing the weights

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

##### Creating the generator

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4 
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

##### Creating the discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // (2 ** 4) # gives 3
        self.adv_layer = nn.Sequential(nn.Linear(128 * 4 ** 2, 1), nn.Sigmoid()) # s'attend à (1152, 1)

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1) # torch.Size([64, 2048])
        validity = self.adv_layer(out)

        return validity

##### Creating the loss functions

In [ ]:

adversarial_loss = torch.nn.BCELoss()


generator = Generator()
discriminator = Discriminator()

In [ ]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [ ]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_laye

##### dataloader


In [ ]:
# Configure data loader
dataloader = torch.utils.data.DataLoader(data,
    batch_size=opt.batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

##### Train the model

In [ ]:
for epoch in range(1, opt.n_epochs + 1):
    for i, (imgs, _) in enumerate(dataloader):
        
       
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

  
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        if not os.path.isdir('output_co'):
            os.mkdir('output_co')
        
        batches_done = epoch * len(dataloader) + i + 1
    
  
        if batches_done % opt.sample_interval == 0:
           save_image(gen_imgs.data[:25], "./output_co/%d.png" % batches_done, nrow=5, normalize=True)

        if epoch % 50 == 0:
         print(
            "[Epoch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, d_loss.item(), g_loss.item())
        )

[Epoch 50/5000] [D loss: 0.322191] [G loss: 1.834512]
[Epoch 50/5000] [D loss: 0.460701] [G loss: 1.116610]
[Epoch 50/5000] [D loss: 0.286840] [G loss: 0.985374]
[Epoch 50/5000] [D loss: 0.608223] [G loss: 1.663139]
[Epoch 50/5000] [D loss: 0.567585] [G loss: 2.529334]
[Epoch 50/5000] [D loss: 0.453574] [G loss: 1.023329]
[Epoch 50/5000] [D loss: 0.477985] [G loss: 0.829671]
[Epoch 50/5000] [D loss: 0.233437] [G loss: 1.783875]
[Epoch 100/5000] [D loss: 0.161954] [G loss: 1.877425]
[Epoch 100/5000] [D loss: 0.159579] [G loss: 1.859585]
[Epoch 100/5000] [D loss: 0.245387] [G loss: 2.367885]
[Epoch 100/5000] [D loss: 0.166793] [G loss: 1.979897]
[Epoch 100/5000] [D loss: 0.425091] [G loss: 2.045609]
[Epoch 100/5000] [D loss: 0.327703] [G loss: 1.049248]
[Epoch 100/5000] [D loss: 0.229337] [G loss: 1.249271]
[Epoch 100/5000] [D loss: 0.657297] [G loss: 2.152554]
[Epoch 150/5000] [D loss: 0.239035] [G loss: 2.740991]
[Epoch 150/5000] [D loss: 0.196315] [G loss: 2.060412]
[Epoch 150/5000] [

##### Save the model

In [ ]:
torch.save(generator.state_dict(), 'generator')
torch.save(discriminator.state_dict(), 'discriminator')

In [ ]:
ls

chest-xray-pneumonia.zip  covid-chestxray-dataset/  inception.h5    patients/
cnn.h5                    discriminator             output_co/      vgg15.h5
co/                       generator                 output_covid19


##### function to save images

In [ ]:
def sample_image(n_row, batches_done):
    z = Variable(Tensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    gen_imgs = generator(z)
    save_image(gen_imgs.data, "./output_co/%d.png" % batches_done, nrow=n_row, normalize=True)

##### Generate pictures

In [ ]:
images = 0

for epoch in range(1, 2_50 + 1): 
    for i, (imgs, _) in enumerate(dataloader, 1):
        
        with torch.no_grad():

            # Adversarial ground truths
            valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
            fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

            # Configure input
            real_imgs = Variable(imgs.type(Tensor))

            batches_done = epoch * len(dataloader) + i
            sample_image(n_row=5, batches_done=batches_done)
            images += 25
            
    if images % 5_000 == 0:
        print(f'Pictures created: {images:,}')
        
    if len(os.listdir(os.path.join(os.getcwd(), 'output_co')))  >= 1_000:
        print('\n images successfully generated.')
        break


 images successfully generated.
